In [15]:
# change your hashcat setting here
# hash type
!echo "0" > /root/hash_type
# attack mode
!echo "0" > /root/attack_mode
# If you have a customized dic, provide the url here. By default, we will use rockyou.txt. Ref: https://github.com/praetorian-inc/Hob0Rules/blob/master/wordlists/rockyou.txt.gz
custom_dic_url = ""
# custome rule set. If you have a customized rule, provide the url here. By default, we will use OneRuleToRuleThemAll (https://github.com/NotSoSecure/password_cracking_rules). Leave empty to not use rule file
custom_rule_url = "OneRuleToRuleThemAll"
# hash needed to crack
crack_it = ""

In [ ]:
# Check GPU Specs
!nvidia-smi

In [ ]:
# Clone, build and install Hashcat
!apt install cmake build-essential -y
!apt install checkinstall git -y
!git clone https://github.com/hashcat/hashcat.git
!cd hashcat && git submodule update --init && make && make install

In [ ]:
# Check the Hashcat Version to make sure it's installed correctly
!hashcat --version

In [ ]:
# WPA2 EAPOL Benchmark (change this to check the speed of whichever hash you're cracking)
!hashcat -w 4 -m 2500 --benchmark

In [5]:
# Get wordlists
!mkdir /root/wordlists && cd /root/wordlists

In [6]:
from IPython import get_ipython

# check custom dic
if custom_dic_url:
  get_ipython().system_raw('wget ' + custom_dic_url + " -O /root/dic.txt")
else:
  get_ipython().system_raw('git clone https://github.com/danielmiessler/SecLists /root/wordlists/SecLists')
  get_ipython().system_raw('tar -xf /root/wordlists/SecLists/Passwords/Leaked-Databases/rockyou.txt.tar.gz -C /root/')
  get_ipython().system_raw('mv /root/rockyou.txt /root/dic.txt')


In [ ]:
!ls -l /root/dic.txt

In [22]:
# check rule file
if custom_rule_url:
  if custom_rule_url == "OneRuleToRuleThemAll":
    get_ipython().system_raw('wget https://raw.githubusercontent.com/NotSoSecure/password_cracking_rules/master/OneRuleToRuleThemAll.rule -O /root/rule.txt')
  else:
    get_ipython().system_raw('wget ' + custom_rule_url + " -O /root/rule.txt")


In [ ]:
!ls -l /root/rule.txt

In [10]:
# Check that your hashes
if crack_it:
  with open("/root/crack.it", "w") as fd:
    fd.write(crack_it)
    fd.close()
else:
  print("WARNING: You Need input your hash to crack")

In [ ]:
!cat /root/crack.it

In [ ]:
# Start pure brute force cracking with a session name
# !hashcat -m 2500 -a3 --session $SESSION /root/.hashcat/hashes/test.hccapx
# !hashcat -m 2500 /root/.hashcat/hashes/test.hccapx -a 3 -w 4 --session test-session

# Restore the session
# !hashcat --session $SESSION --restore

# Crack with a wordlist
!ls -l /root/rule.txt
!if [ $? == 0 ]; then hashcat -m `cat /root/hash_type` -a `cat /root/attack_mode` /root/crack.it /root/dic.txt -r /root/rule.txt; hashcat -m `cat /root/hash_type` -a `cat /root/attack_mode` /root/crack.it /root/dic.txt -r /root/rule.txt --show;  else hashcat -m `cat /root/hash_type` -a `cat /root/attack_mode` /root/crack.it /root/dic.txt -r /root/rule.txt; hashcat -m `cat /root/hash_type` -a `cat /root/attack_mode` /root/crack.it /root/dic.txt --show; fi